In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

ua = UserAgent()

In [2]:
headers = {
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
'cache-control': 'max-age=0',
'cookie': 'perseusRolloutSplit=2; dhhPerseusSessionId=1572777846270.344129640382112400.tqve6035lc8; dhhPerseusGuestId=1572773675054.296431960063008960.th00zfvm5m; __cfduid=d49830dd8faef655d8fd7859443a1fd1a1572773671; hl=en; AppVersion=02c2d36; ld_key=118.100.76.150; _pxhd=6097c3582e3b323a10236581cc2f09942a843225042959a6592317fbc32efa2c:23bbaec1-fe1d-11e9-b8fd-5d9da8797a65; optimizelyEndUserId=oeu1572773673643r0.3029206072281285; _pxvid=23bbaec1-fe1d-11e9-b8fd-5d9da8797a65; perseusRolloutSplit=2; dhhPerseusGuestId=1572773675054.296431960063008960.th00zfvm5m; _gcl_au=1.1.1974466263.1572773675; _ga=GA1.2.834634854.1572773675; _gid=GA1.2.733629929.1572773675; _hjid=cbf68d6c-0e9a-4343-87da-bdb6dd825dc6; __ssid=b43de24b73336ad349e495d33424f8e; ViewContentStatus=true; _dc_gtm_UA-90537345-1=1; _dc_gtm_UA-50811932-30=1; _px3=3c729ac794d8d45c7fe754cbd67790d71474a785a2627fe57b8dd1354665be4e:fNXD1m3oMRkZmH5yzruS63pfq2wij+y7tLFTLJITIoz/x88BJDR5YkmH2xWELbrnp0cfUmp+8OXMnJIjFHB8Qw==:1000:AJqIWGYwqD+LpejoGvA2HPJbAfvzQ+jAwi0RQCe+zvgkpfJaiErJeIMGRrPiJG7PVkepT8hmntejklhnnxF8qR57jOzmSCAwb5kSaOuaqH2+u7jScpM+KnlZgmozgyUaqj6QccbWsJ1FO2bNoyzvj//TsuWcVj6EeyYubyxqnQ4=; _tq_id.TV-81365445-1.cf54=fc76d66efbbf4a71.1572773676.0.1572779104..',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

In [3]:
# headers = {"User-Agent": ua.chrome, 
#            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
#            "Accept-Language": "en-US,en;q=0.5", 
#            "Accept-Encoding": "gzip, deflate", "DNT": "1", "Connection": "close", 
#            "Upgrade-Insecure-Requests": "13434"}
cookies = {'JSESSIONID': 'fbadab78eef7d2d'}

In [4]:
import json

with open('foodpanda-restaurant.json') as fopen:
    restaurants = json.load(fopen)
    
restaurants.keys()

dict_keys(['Kuala Lumpur', 'Penang', 'Petaling Jaya', 'Subang', 'Shah Alam', 'Cyberjaya', 'Johor Bahru', 'Melaka', 'Kuching', 'Miri', 'Kuantan', 'Bangi', 'Ipoh', 'Terengganu', 'Kajang', 'Kelantan', 'Putrajaya', 'Kota-Kinabalu', 'Klang', 'Negeri Sembilan', 'Sungai Petani', 'Alor Setar', 'Kulim', 'Rawang', 'Taiping', 'Sitiawan', 'Kluang', 'Kulai', 'Sibu', 'Sepang', 'Kangar', 'Temerloh', 'Bintulu', 'Petra Jaya', 'Sandakan', 'Pontian', 'Segamat', 'Nibong Tebal', 'Kampar', 'Ulu Tiram'])

In [7]:
restaurants['Kuala Lumpur']['Husen Cafe']

{'star': '4.3',
 'delivery': 'Free',
 'characters': ['Meat', 'Noodles', 'Rice', 'Local Champions', 'Indian'],
 'link': '/restaurant/n1br/husen-cafe'}

In [8]:
import time

parsed = {}

In [23]:
quit = False

for no, (city, city_values) in enumerate(list(restaurants.items())):
    if quit:
        break
        
    for no_nested, (restaurant, restaurant_values) in enumerate(list(city_values.items())):
        if '%s <> %s'%(city, restaurant) in parsed:
            print('%s <> %s'%(city, restaurant), ' already inside parsed')
            continue
    
        try:
            response = requests.get('https://www.foodpanda.my' + restaurant_values['link'], 
                        headers = headers, cookies = cookies, timeout = 60)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            if 'Access to this page has been denied' in soup.title.contents[0]:
                quit = True
                print('quit at ', no, no_nested)
                break
                
            foods = soup.find_all("div", class_="dish-card h-product menu__item")
            
            print('PARSING ', restaurant)
            data = []
            for food in foods:
                try:
                    data.append([food.get('data-menu-category'), json.loads(food.get('data-object'))])
                except:
                    print('ERROR 404 food ', restaurant)
            parsed['%s <> %s'%(city, restaurant)] = data
            restaurants[city][restaurant]['data'] = data
            
            
        except:     
            print('ERROR 404 ', restaurant)

Kuala Lumpur <> La Risata Bar Pizzeria Ristorante  already inside parsed
Kuala Lumpur <> Viapre Italian Restaurant KL  already inside parsed
Kuala Lumpur <> Sri Ayutthaya  already inside parsed
Kuala Lumpur <> Victoria Station  already inside parsed
Kuala Lumpur <> Passage Thru India  already inside parsed
Kuala Lumpur <> Jaya Suasana Enterprise (Kolej Kediaman 11)  already inside parsed
Kuala Lumpur <> OhJuice Malaysia (Universiti Malaya)  already inside parsed
Kuala Lumpur <> Famous Fried Chicken (UM - Fakulti 12)  already inside parsed
Kuala Lumpur <> Symphony by Chef Jo  already inside parsed
Kuala Lumpur <> Bangsar Venicia  already inside parsed
Kuala Lumpur <> Fuel Shack  already inside parsed
Kuala Lumpur <> Ali Maju  already inside parsed
Kuala Lumpur <> Champs  already inside parsed
Kuala Lumpur <> Just Heavenly Cafe  already inside parsed
Kuala Lumpur <> Madam Kwan  already inside parsed
Kuala Lumpur <> Chili's Grill & Bar  already inside parsed
Kuala Lumpur <> Yusuf Tom Yam 

In [20]:
len(parsed)

4697

In [21]:
restaurants['Kuala Lumpur']['Husen Cafe']

{'star': '4.3',
 'delivery': 'Free',
 'characters': ['Meat', 'Noodles', 'Rice', 'Local Champions', 'Indian'],
 'link': '/restaurant/n1br/husen-cafe',
 'data': [['Rojak',
   {'is_half_type_available': False,
    'id': 1437339,
    'name': 'Rojak',
    'code': None,
    'description': None,
    'file_path': '',
    'logo_path': '',
    'half_type': None,
    'is_alcoholic_item': False,
    'product_variations': [{'id': 1800870,
      'code': None,
      'name': None,
      'price': 5.8,
      'price_before_discount': None,
      'container_price': 0,
      'choices': [],
      'toppings': [],
      'topping_ids': []}],
    'master_category_id': 3,
    'is_prepacked_item': False,
    'exclude_dish_information': False,
    'has_dish_information': False,
    'is_sold_out': False,
    'additives': [],
    'sold_out_options': []}],
  ['Rojak',
   {'is_half_type_available': False,
    'id': 1437340,
    'name': 'Rojak Mee',
    'code': None,
    'description': None,
    'file_path': '',
    'l

In [22]:
import json

with open('foodpanda-foods.json', 'w') as fopen:
    json.dump(restaurants, fopen)